In [42]:
!pip install \
    --trusted-host pypi.org \
    --trusted-host files.pythonhosted.org \
    networkx numpy matplotlib
print("____cell1 run_____")

____cell1 run_____


In [43]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import warnings

#to display the plot in the jupyter code block itself 
%matplotlib inline 

plt.rcParams['figure.figsize'] = (10, 7)  #fixing configurations for the rest of the map
warnings.filterwarnings('ignore')
print("______Cell2 ran_____")

______Cell2 ran_____


In [44]:
G = nx.karate_club_graph()  #Loads Karate club graph

print("Graph Info:")
print(f"Nodes: {G.number_of_nodes()}")
print(f"Edges: {G.number_of_edges()}")


pos = nx.spring_layout(G, seed=42)  #to fix the cordinate of the nodes so that they dont 'jump around' for each iteration
print("________Cell3 run_______")

Graph Info:
Nodes: 34
Edges: 78
________Cell3 run_______


In [45]:
#function for modularity matrix:

def modularity_matrix(G):
    n = G.number_of_nodes()
    m = G.number_of_edges()

    if m == 0:
        return np.zeros((n,n))
    
    nodelist = sorted(G.nodes()) # gets sorted list of nodes in the grpah G

    A = nx.to_numpy_array(G, nodelist)  # gets the adjacency matrix
     
    k = A.sum(axis=1) # to get sum along axis 1, that is the row of the adjaceny matrix; therefore gets the degree of each node
    

    expected_edges = np.outer(k, k) / (2 * m)
    #outer product is a matrix where each entry is the product of an element from the first vector and an element from the second vector
    #hence it gives the same required result like multiplying the vector and its transpose
    
    
    B = A - expected_edges # modularity matrix is calculated here
    
    return B
print("_________cell4 ran________")

_________cell4 ran________


In [47]:
def find_communities(G):
    
# to perform iterative spectral bisection 
    
    
    B_global = modularity_matrix(G)
    
    all_nodes_list = sorted(G.nodes()) 
    node_indices = {node: i for i, node in enumerate(all_nodes_list)}  # maps each current node to its index(position) in the orignal global B
    
    queue = [all_nodes_list.copy()] #contains all the communities to be processed ; starts with having all the nodes
    
    final_communities = [] # list of all communities with negative eigen  value i.e. indivisible communities
    
   
    partition_history = []
    metric_history = []
    
    iteration = 0
    
    
    while queue:     # looped as long there the queue becomes empty
        
        
        current_partition = final_communities.copy() + queue.copy() # The partition at this iteration is all final groups + all groups in the queue
        partition_history.append(current_partition)
        
        iter_metrics = {
            'degree': {},
            'betweenness': {},
            'closeness': {},
            'clustering': {}
        }
        
        for C in current_partition:
            sub_G = G.subgraph(C)
            n_sub = len(C)
        #iterates through all the communities in the current partition to calculate the meterics
            
            if n_sub > 1:
                #to exclude single node entries
                deg = nx.degree_centrality(sub_G)  #count of how many edges (connections) a node has.
                bet = nx.betweenness_centrality(sub_G)  #A measure of how often a node lies on the shortest path between other pairs of nodes in the network.; 
                                                        #broker/bridge
                
                if nx.is_connected(sub_G):    #closeness is only defined for connected graphs
                    clo = nx.closeness_centrality(sub_G)  #A measure of how fast a node can reach every other node in the network. 
                                                          #It's based on the average of its shortest path distances to all other nodes.
                else:
                    clo = {node: 0.0 for node in C}  # else assign 0 if the subgraph is not connected
                    
                clu = nx.clustering(sub_G)         #estimate of how for a node, how well its neighboouring nodes are connected; checking for a closely knit group
                
                # Update the metrics dictionary
                iter_metrics['degree'].update(deg)
                iter_metrics['betweenness'].update(bet)
                iter_metrics['closeness'].update(clo)
                iter_metrics['clustering'].update(clu)
                
            elif n_sub == 1:
                # Metrics for a single-node graph are 0
                node = C[0]
                iter_metrics['degree'][node] = 0.0
                iter_metrics['betweenness'][node] = 0.0
                iter_metrics['closeness'][node] = 0.0
                iter_metrics['clustering'][node] = 0.0
        
        metric_history.append(iter_metrics)
        
    
        if not queue:
            break      # if the queue is empty , all possible communities are split , thus break out of the loop
            
    
        nodes_to_split = queue.pop(0)     # Get first community in queue 
        
        
        if len(nodes_to_split) <= 1:  # if only one node is present, no possible splitting , thus its final
            final_communities.append(nodes_to_split)
            continue
            

        
        indices = [node_indices[node] for node in nodes_to_split]    # Get the indices in B_global corresponding to our community
        
        B_restricted = B_global[np.ix_(indices, indices)]   #selects only the indices of our interest (from current partition) from the global modularity matrix
                                                            #thus creates a sub matirx
        
        
        eigenvalues, eigenvectors = np.linalg.eigh(B_restricted)  # gets eigenvalue and eigenvector 
        
        
        lambda_1 = eigenvalues[-1]  # np.linalg.eigh() sorts eigenvalue in ascending order; so to get the largest eigenvalue , we need to take the last element
        u_1 = eigenvectors[:, -1]   # to get the corresponding eigenvector
        
        # to stop iterations
        if lambda_1 <= 1e-10:    # No positive eigenvalue, so no split improves modularity
            final_communities.append(nodes_to_split)

        
        else:
             #assigns each node to a group according to the value of the leading eigen value component corresponding to that node
            C_plus = []   
            C_minus = []
            for i, node in enumerate(nodes_to_split):
                
                if u_1[i] > 0:        # Assign to communities based on sign of eigenvector 
                    C_plus.append(node)
                else:                # Assign to communities based on sign of eigenvector
                    C_minus.append(node)
            
            # Add the two new sub-communities back to the queue, for them to be examined again for next iteration
            
            if C_plus:
                queue.append(C_plus)
            if C_minus:
                queue.append(C_minus)
        
        iteration += 1

    return partition_history, metric_history

print("___________cell5 ran___________")

___________cell5 ran___________
